In [1]:
import pandas as pd
import numpy as np
import csv
import os 
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import math
from Functions import nb_gamesH2H, nb_victoryH2H,prct_victoryH2H
all_data= pd.read_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\all_data_ml.csv', error_bad_lines=False, sep =',')
pd.set_option('max_columns', 90)


In [6]:
def h2h(name1, name2,df):
    """ A slow way to code H2H """
    
    V1=0
    V2=0
    for i in range(len(df['player_1_name'])):
        if (df['player_1_name'].iloc[i]==name1 and df['player_2_name'].iloc[i]==name2):
            if df['player_1_win'].iloc[i]==1:
                V1=V1+1
            else:
                V2=V2+1
        if (df['player_1_name'].iloc[i]==name2 and df['player_2_name'].iloc[i]==name1):
            if df['player_1_win'].iloc[i]==1:
                V2=V2+1
            else:
                V1=V1+1
    print(V1)
    print(V2)
    if V1+V2>0:
        return [V1+V2,(V1/(V1+V2))]
    return False


In [ ]:
# WARNING: This file is only meant to generate the database(3/3) do not run except if you don't have the generated database.

In [67]:
# Construction of the last feature Head to Head

lt_H2H = []
for i in tqdm(range(len(all_data[0:100]))):
    lt_H2H.append(prct_victoryH2H(all_data["player_1_name"].loc[i],all_data["player_2_name"].loc[i], all_data))
lt_H2H
all_data.insert(1, "H2H_player_1_vs_player_2", lt_H2H, allow_duplicates = False)

100%|████████████████████████████████████████████████████████████████████████| 109844/109844 [5:43:28<00:00,  5.33it/s]


In [ ]:
del all_data['Unnamed: 0']

In [101]:
data_ML = all_data[['player_1_id','player_1_win','player_2_id','id_tourney','id_round','date_tourney','player_1_name',
                   'player_2_name','surface','player_1_odds', 'player_2_ods', 'player_1_diff_fs_prct','player_1_diff_pt_fs','player_1_diff_ss_prct',
                   'player_1_diff_def','player_1_diff_bp','player_1_diff_wsp','player_1_serveadv_diff','player_1_aces_diff_pcrt',
                   'player_1_diff_df_pcrt', 'diff_age', 'player_1_diff_completeness', 'win_ratio_diff','H2H_player_1_vs_player_2',
                   ]]
data_CO = all_data[['player_1_id','surface','player_1_win','player_2_id','id_tourney','id_round','date_tourney','player_1_name',
                   'player_2_name','player_1_odds', 'player_2_ods', 'player_1_fs_prct', 'player_2_fs_prct','player_1_ss_prct', 'player_2_ss_prct',
                    'player_1_def_prct', 'player_2_def_prct','player_1_bp_prct', 'player_2_bp_prct','player_1_wsp', 'player_2_wsp'
                    ,'player_1_completeness', 'player_2_completeness', 'player_1_serveadv','player_2_serveadv',
                    'player_1_aces_pcrt','player_2_aces_pcrt','player_2_df_pcrt','player_1_df_pcrt','player_1_fs_pt_prct', 'player_2_fs_pt_prct',
                    
                   ]]
'''
We divide the features of the main dataset into two datasets, one for optimizing the logistivc regression and the SVM
data_ML will be used for optimizing the logistic regression function, there is only the features labelled 'diff' in it
On the other hand data_CO will be used to 'invent' the matches that don't have take place yet, in order 
to 'feed' out logistic regression function.
'''

In [102]:
    # Get the years so we can split the data
years = all_data["date_tourney"]
years = pd.Series([y[0:4] for y in years.values])

    # Split into train, validation, and test bydate
    # [2004, 2017] = 60% of data
    # [2018, 2019] = 20% of data
    # [2019, 2020] = % of data
train_shape = years[(years >= '2004') & (years <= '2017')].shape[0]
valid_shape = years[(years >= '2018') & (years <= '2019')].shape[0]
test_shape = years[(years >= '2020')].shape[0]

# Partition
x_sample_ML = data_ML[0:1000]
x_train_ML = data_ML[0:train_shape]
x_valid_ML = data_ML[train_shape: train_shape + valid_shape]
x_test_ML = data_ML[train_shape + valid_shape: train_shape + valid_shape + test_shape]

    # Shuffle
x_train_ML = x_train_ML.sample(frac=1).reset_index(drop=True)
x_valid_ML = x_valid_ML.sample(frac=1).reset_index(drop=True)
x_test_ML = x_test_ML.sample(frac=1).reset_index(drop=True)
# Save the splits
all_data.to_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\all_data_ml_co.csv', encoding="utf8")
x_train_ML.to_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\data_train_ml.csv', encoding="utf8")
x_valid_ML.to_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\data_valid_ml.csv', encoding="utf8")
x_test_ML.to_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\data_test_ml.csv', encoding="utf8")
x_sample_ML.to_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\data_sample_ml.csv', encoding="utf8")

In [103]:
# Partition
x_sample_CO = data_CO[0:1000]
x_train_CO = data_CO[0:train_shape]
x_valid_CO = data_CO[train_shape: train_shape + valid_shape]
x_test_CO = data_CO[train_shape + valid_shape: train_shape + valid_shape + test_shape]

    # Shuffle
x_train_CO = x_train_CO.sample(frac=1).reset_index(drop=True)
x_valid_CO = x_valid_CO.sample(frac=1).reset_index(drop=True)
x_test_CO = x_test_CO.sample(frac=1).reset_index(drop=True)
# Save the splits
data_CO.to_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\all_data_co.csv', encoding="utf8")
x_train_CO.to_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\data_train_co.csv', encoding="utf8")
x_valid_CO.to_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\data_valid_co.csv', encoding="utf8")
x_test_CO.to_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\data_test_co.csv', encoding="utf8")
x_sample_CO.to_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\data_sample_co.csv', encoding="utf8")